In [42]:
import os
from nltk.corpus import stopwords
import string
import pandas as pd

In [43]:
STOPS = stopwords.words("english")
EXCLUDE = set(string.punctuation) | set(["''", "BR", "--", "/td", "nbsp", "2002", "localhost", "&nbsp"])
SPAMPATH = "./spam/"
HAMPATH = "./easy_ham/"

In [44]:
STOPS.extend(['delivery-date', 'jalapeno', 'return-path', 'delivered-to', 'received', 'esmtp', 'smtp'])
STOPS.extend(['message-id', 'imap', 'date', 'subject', 'list-help', 'list-post', 'list-subscribe', 'list-archive'])
STOPS.extend(['x-beenthere', 'mime-version', 'errors-to', 'list-unsubscribe', 'x-mailman-version'])
STOPS.extend(['content-type', 'list-id', 'dogmaslashnullorg', 'xentcom', 'in-reply-to'])
STOPS.extend(['jmasonorg', 'sender', 'bulk', 'content-transfer-encoding'])
STOPS.extend(['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'])
STOPS.extend(['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat'])

In [45]:
def ListToString(inputlist):
    cleanstring = ""
    for each in inputlist:
        cleanstring = cleanstring + each + " "
    return cleanstring.strip()

In [46]:
def FilterWord(s):
    """
    Receives a string and determines whether or not it is a string that should be used
    for analysis (keep = 1) and passes back a clean version of the string.
    """
    keep = 1 #default is to keep the word
    cleanstring = s.lower()
    drop_chars = ",.:;!"
    for c in drop_chars:
        cleanstring = cleanstring.replace(c, "")
    bad_chars = "><#$/=0123456789+()&*@?"
    for c in bad_chars:
        if cleanstring.find(c)!=-1:
            keep=0
    if len(cleanstring)<3:
        keep=0
    if cleanstring in STOPS:
        keep =0
    if cleanstring in EXCLUDE:
        keep=0
    return keep, cleanstring

In [47]:
def CleanWords(wordlist):
    """
    Process go go through a list of words and clean each of them up.  Passes back
    a list of clean words.
    """
    filtered_words =[]
    for each in wordlist:
        flag, cleanstring = FilterWord(each)
        if flag==1:
            filtered_words.append(cleanstring)
    return filtered_words

In [48]:
df = pd.DataFrame(columns=['id', 'spam', 'file', 'words'])
count = 0
for (dirpath, dirnames, filenames) in os.walk(SPAMPATH):
    for file in filenames:
        infile=(SPAMPATH + file)
        f = open(infile, 'r', encoding="Latin-1")
        s = f.read()
        f.close()
        #s = ''.join(filter(lambda x: x in string.printable, s))
        #wordlist=s.split()
        words = s.split()
        words = CleanWords(words)
        cleanstring = ListToString(words)
        # print(len(words))
        # print(words)
        df = df.append({'id':count, 'spam':1, 'file':file, 'words':cleanstring}, ignore_index=True)
        count = count+1
for (dirpath, dirnames, filenames) in os.walk(HAMPATH):
    for file in filenames:
        infile=(HAMPATH + file)
        f = open(infile, 'r', encoding="Latin-1")
        s = f.read()
        f.close()
        #s = ''.join(filter(lambda x: x in string.printable, s))
        #wordlist=s.split()
        words = s.split()
        words = CleanWords(words)
        cleanstring = ListToString(words)
        # print(len(words))
        # print(words)
        df = df.append({'id':count, 'spam':0, 'file':file, 'words':cleanstring}, ignore_index=True)
        count = count+1

In [49]:
df.to_csv('email_corpus.csv')